# Custom Planning Multi-Agent System  

In this notebook, we will explore how to propmt an LLM to write, refine, and follow a plan to generate a report using multiple agents.  

This is not meant to be a comprehensive guide to creating a report generation system, but rather, giving you the knowledge and tools to build your own robust systems that can plan and orchestrate multiple agents to achieve a goal.  

This notebook will assume that you have already either read the [basic agent workflow notebook](https://docs.llamaindex.ai/en/stable/examples/agent/agent_workflow_basic/) or the [agent workflow documentation](https://docs.llamaindex.ai/en/stable/understanding/agent/), as well as the [workflow documentation](https://docs.llamaindex.ai/en/stable/understanding/workflows/). 

## Setup  

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

from llama_index.llms.huggingface import HuggingFaceLLM

/home/user/anaconda3/envs/2hg_llamaindex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sub_agent_llm = HuggingFaceLLM(
    model_name="/media/user/datadisk2/LLM_models/Qwen2.5-7B-Instruct-1M",
    tokenizer_name="/media/user/datadisk2/LLM_models/Qwen2.5-7B-Instruct-1M",
    device_map="auto",
    max_new_tokens=8192
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.53s/it]


## System Design  

Our system will have three agents:  
1. A `ResearchAgent` that will search the web for information on the given topic.  
2. A `WriteAgent` that will write the report using the information found by the `ResearchAgent`.  
3. A `ReviewAgent` that will review the report and provide feedback.  

We will then use a top-level LLM to manually orchestrate and plan around the other agents to write our report.  

While there are many ways to implement this system, in this case, we will use a single `web_search` tool to search the web for information on the given topic.  

In [3]:
from tavily import AsyncTavilyClient
from config import TAVILY_API_KEY

async def search_web(query: str) -> str:
    """Useful for using the web to answer questions."""
    client = AsyncTavilyClient(api_key=TAVILY_API_KEY)
    return str(await client.search(query))

With our tool defined, we can now create our sub-agents.  

If the LLM you are using supports tool calling, you can use the `FunctionAgent` class.  

Otherwise, you can use the `ReActAgent` class.

In [4]:
from llama_index.core.agent.workflow import FunctionAgent, ReActAgent

research_agent = ReActAgent(
    name="ResearchAgent",
    description="Useful for recording research notes based on a specific prompt.",
    system_prompt=(
        "You are the ResearchAgent that can search the web for information on a given topic and record notes on the topic. "
        "You should output notes on the topic in a structured format."
    ),
    llm=sub_agent_llm,
    tools=[search_web],
)

write_agent = ReActAgent(
    name="WriteAgent",
    description="Useful for writing a report based on the research notes or revising the report based on feedback.",
    system_prompt=(
        "You are the WriteAgent that can write a report on a given topic. "
        "Your report should be in a markdown format. The content should be grounded in the research notes. "
        "Return your markdown report surrounded by ... tags."
    ),
    llm=sub_agent_llm,
    tools=[],
)

review_agent = ReActAgent(
    name="ReviewAgent",
    description="Useful for reviewing a report and providing feedback.",
    system_prompt=(
        "You are the ReviewAgent that can review the write report and provide feedback. "
        "Your review should either approve the current report or request changes to be implemented."
    ),
    llm=sub_agent_llm,
    tools=[],
)

With earch agent defined, we can also write helper functions to help execute each agent.

In [5]:
import re
from llama_index.core.workflow import Context


async def call_research_agent(ctx: Context, prompt: str) -> str:
    """Useful for recording research notes based on a specific prompt."""
    result = await research_agent.run(
        user_msg=f"Write some notes about the following: {prompt}"
    )

    state = await ctx.store.get("state")
    state["research_notes"].append(str(result))
    await ctx.store.set("state", state)

    return str(result)


async def call_write_agent(ctx: Context) -> str:
    """Useful for writing a report based on the research notes or revising the report based on feedback."""
    state = await ctx.store.get("state")
    notes = state.get("research_notes", None)
    if not notes:
        return "No research notes to write from."

    user_msg = f"Write a markdown report from the following notes. Be sure to output the report in the following format: ...:\n\n"

    # Add the feedback to the user message if it exists
    feedback = state.get("review", None)
    if feedback:
        user_msg += f"{feedback}\n\n"

    # Add the research notes to the user message
    notes = "\n\n".join(notes)
    user_msg += f"{notes}\n\n"

    # Run the write agent
    result = await write_agent.run(user_msg=user_msg)
    report = re.search(r"(.*)", str(result), re.DOTALL).group(
        1
    )
    state["report_content"] = str(report)
    await ctx.store.set("state", state)

    return str(report)


async def call_review_agent(ctx: Context) -> str:
    """Useful for reviewing the report and providing feedback."""
    state = await ctx.store.get("state")
    report = state.get("report_content", None)
    if not report:
        return "No report content to review."

    result = await review_agent.run(
        user_msg=f"Review the following report: {report}"
    )
    state["review"] = result
    await ctx.store.set("state", state)

    return result

## Defining the Planner Workflow  

In order to plan around the other agents, we will write a custom workflow that will explicitly orchestrate and plan the other agents.  

Here our prompt assumes a sequential plan, but we can expand it in the future to support parallel steps. (This just involves more complex parsing and prompting, which is left as an exercise for the reader.)

In [6]:
import re
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Any, Optional

from llama_index.core.llms import ChatMessage
from llama_index.core.workflow import (
    Context,
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

PLANNER_PROMPT = """You are a planner chatbot. 

Given a user request and the current state, break the solution into ordered <step> blocks.  Each step must specify the agent to call and the message to send, e.g.
<plan>
  <step agent=\"ResearchAgent\">search for …</step>
  <step agent=\"WriteAgent\">draft a report …</step>
  ...
</plan>

<state>
{state}
</state>

<available_agents>
{available_agents}
</available_agents>

The general flow should be:
- Record research notes
- Write a report
- Review the report
- Write the report again if the review is not positive enough

If the user request does not require any steps, you can skip the <plan> block and respond directly.
"""


class InputEvent(StartEvent):
    user_msg: Optional[str] = Field(default=None)
    chat_history: list[ChatMessage]
    state: Optional[dict[str, Any]] = Field(default=None)


class OutputEvent(StopEvent):
    response: str
    chat_history: list[ChatMessage]
    state: dict[str, Any]


class StreamEvent(Event):
    delta: str


class PlanEvent(Event):
    step_info: str


# Modelling the plan
class PlanStep(BaseModel):
    agent_name: str
    agent_input: str


class Plan(BaseModel):
    steps: list[PlanStep]


class ExecuteEvent(Event):
    plan: Plan
    chat_history: list[ChatMessage]


class PlannerWorkflow(Workflow):
    llm: HuggingFaceLLM = HuggingFaceLLM(
        model_name="/media/user/datadisk2/LLM_models/Qwen2.5-14B-Instruct-1M",
        tokenizer_name="/media/user/datadisk2/LLM_models/Qwen2.5-14B-Instruct-1M",
        device_map="auto",
        max_new_tokens=8192
    )
    agents: dict[str, FunctionAgent] = {
        "ResearchAgent": research_agent,
        "WriteAgent": write_agent,
        "ReviewAgent": review_agent,
    }

    @step
    async def plan(
        self, ctx: Context, ev: InputEvent
    ) -> ExecuteEvent | OutputEvent:
        # Set initial state if it exists
        if ev.state:
            await ctx.store.set("state", ev.state)

        chat_history = ev.chat_history

        if ev.user_msg:
            user_msg = ChatMessage(
                role="user",
                content=ev.user_msg,
            )
            chat_history.append(user_msg)

        # Inject the system prompt with state and available agents
        state = await ctx.store.get("state")
        available_agents_str = "\n".join(
            [
                f'<agent name="{agent.name}">{agent.description}</agent>'
                for agent in self.agents.values()
            ]
        )
        system_prompt = ChatMessage(
            role="system",
            content=PLANNER_PROMPT.format(
                state=str(state),
                available_agents=available_agents_str,
            ),
        )

        # Stream the response from the llm
        response = await self.llm.astream_chat(
            messages=[system_prompt] + chat_history,
        )
        full_response = ""
        async for chunk in response:
            full_response += chunk.delta or ""
            if chunk.delta:
                ctx.write_event_to_stream(
                    StreamEvent(delta=chunk.delta),
                )

        # Parse the response into a plan and decide whether to execute or output
        xml_match = re.search(r"(<plan>.*</plan>)", full_response, re.DOTALL)

        if not xml_match:
            chat_history.append(
                ChatMessage(
                    role="assistant",
                    content=full_response,
                )
            )
            return OutputEvent(
                response=full_response,
                chat_history=chat_history,
                state=state,
            )
        else:
            xml_str = xml_match.group(1)
            root = ET.fromstring(xml_str)
            plan = Plan(steps=[])
            for step in root.findall("step"):
                plan.steps.append(
                    PlanStep(
                        agent_name=step.attrib["agent"],
                        agent_input=step.text.strip() if step.text else "",
                    )
                )

            return ExecuteEvent(plan=plan, chat_history=chat_history)

    @step
    async def execute(self, ctx: Context, ev: ExecuteEvent) -> InputEvent:
        chat_history = ev.chat_history
        plan = ev.plan

        for step in plan.steps:
            agent = self.agents[step.agent_name]
            agent_input = step.agent_input
            ctx.write_event_to_stream(
                PlanEvent(
                    step_info=f'<step agent="{step.agent_name}">{step.agent_input}</step>'
                ),
            )

            if step.agent_name == "ResearchAgent":
                await call_research_agent(ctx, agent_input)
            elif step.agent_name == "WriteAgent":
                # Note: we aren't passing the input from the plan since
                # we're using the state to drive the write agent
                await call_write_agent(ctx)
            elif step.agent_name == "ReviewAgent":
                await call_review_agent(ctx)

        state = await ctx.store.get("state")
        chat_history.append(
            ChatMessage(
                role="user",
                content=f"I've completed the previous steps, here's the updated state:\n\n\n{state}\n\n\nDo you need to continue and plan more steps?, If not, write a final response.",
            )
        )

        return InputEvent(
            chat_history=chat_history,
        )

Loading checkpoint shards: 100%|██████████| 8/8 [00:20<00:00,  2.56s/it]


## Running the Workflow  

With our custom planner defined, we can now run the workflow and see it in action!  

As the workflow is running, we will stream the events to get an idea of what is happening under the hood.


In [7]:
planner_workflow = PlannerWorkflow(timeout=None)

handler = planner_workflow.run(
    user_msg=(
        "Write me a report on the history of the internet. "
        "Briefly describe the history of the internet, including the development of the internet, the development of the web, "
        "and the development of the internet in the 21st century."
    ),
    chat_history=[],
    state={
        "research_notes": [],
        "report_content": "Not written yet.",
        "review": "Review required.",
    },
)

current_agent = None
current_tool_calls = ""
async for event in handler.stream_events():
    if isinstance(event, PlanEvent):
        print("Executing plan step: ", event.step_info)
    elif isinstance(event, ExecuteEvent):
        print("Executing plan: ", event.plan)

result = await handler

Executing plan step:  <step agent="ResearchAgent">search for the history of the internet, focusing on its development, the development of the web, and its development in the 21st century.</step>
Executing plan step:  <step agent="WriteAgent">draft a report using the research notes provided, covering the history of the internet, the development of the web, and the development of the internet in the 21st century.</step>
Executing plan step:  <step agent="ReviewAgent">review the drafted report on the history of the internet for accuracy and clarity.</step>
Executing plan step:  <step agent="WriteAgent">revise the report based on the feedback from the ReviewAgent, ensuring it briefly describes the history of the internet, including the development of the internet, the development of the web, and the development of the internet in the 21st century.</step>


In [8]:
print(result.response)

Based on the provided information, the report on the history of the internet has already been written, reviewed, and finalized. Here’s the final response:

---

# The History of the Internet and Its Evolution Through the 21st Century

## Conceptualization and Early Development

The Internet was initially conceptualized in the late 1960s by researchers at the United States Department of Defense (DOD) as a decentralized network to ensure communication among military installations in case of a nuclear attack. The first successful message sent over the ARPANET, a precursor to the Internet, happened on October 29, 1969.

## Pivotal Moment: The Invention of the World Wide Web

The World Wide Web (WWW), a system of interlinked hypertext documents, was invented by British computer scientist Tim Berners-Lee in 1989 while working at CERN. The first website went live on August 6, 1991. The web made the Internet accessible to the general public, which led to its rapid growth and popularization.

#

Now, we can retrieve the final report in the system for ourselves.

In [9]:
state = await handler.ctx.get("state")
print(state["report_content"])

Thought: I can answer without using any more tools. I'll use the user's language to answer

```
# The History of the Internet and Its Evolution Through the 21st Century

## Conceptualization and Early Development

The Internet was initially conceptualized in the late 1960s by researchers at the United States Department of Defense (DOD) as a decentralized network to ensure communication among military installations in case of a nuclear attack. The first successful message sent over the ARPANET, a precursor to the Internet, happened on October 29, 1969.

## Pivotal Moment: The Invention of the World Wide Web

The World Wide Web (WWW), a system of interlinked hypertext documents, was invented by British computer scientist Tim Berners-Lee in 1989 while working at CERN. The first website went live on August 6, 1991. The web made the Internet accessible to the general public, which led to its rapid growth and popularization.

## Integration Into Daily Life

In the early 21st century, the int

/tmp/ipykernel_72027/1737397799.py:1: DeprecationWarning: Context.get() is deprecated. Use 'await ctx.store.get()' instead.
  state = await handler.ctx.get("state")


In [10]:
print(state["review"])

Based on the provided report, the history of the Internet and its evolution through the 21st century has been fascinating and transformative. Initially conceived by the U.S. Department of Defense in the late 1960s as a means to maintain communication during a nuclear attack, the Internet has evolved significantly. The invention of the World Wide Web by Tim Berners-Lee in 1989 was a pivotal moment, making the Internet accessible to the general public and leading to its rapid growth and popularization. 

In the early 21st century, the internet has become an integral part of daily life, influencing various aspects including communication, commerce, education, and entertainment. Social media platforms and e-commerce have emerged, connecting people globally and enabling real-time communication. These advancements have created new opportunities and transformed industries. 

In the mid-21st century, the internet continues to evolve with advancements in artificial intelligence, big data, cloud

참고  
- [Custom Planning Multi-Agent System](https://docs.llamaindex.ai/en/stable/examples/agent/custom_multi_agent/)